# Trying to guess DD-finite equations

In [51]:
from ore_algebra import OreAlgebra, guess
from ajpastor.dd_functions import *

OA.<Dx> = OreAlgebra(QQ[x])
def guess_DDFinite(data, order_bound=5, ore_ring=OA):
    r'''
        Method to guess a DD-finite equation from some data
        
        This method tries to compute a differential equation whose coefficients are D-finite 
        functions (i.e., the annihilator for a DD-finite function) starting with the sequence 
        of the function itself.
        
        The output of this method is the actual DD-finite function that have ``data`` as 
        first elements in its sequence, or a :class:`ValueError` if no relation was found.
        
        INPUT:
          - ``data``: any input (list, data or polynomial) that can be casted into a polynomial
            using the base ring of ``ore_ring``.
          - ``order_bound``: a limit for the search of a DD-finite differential equation. We start
            by looking for equations of order 2 and we increase up to this value until we find something.
            ``ore_ring``: the :class:`OreAlgebra` ring that will be used for the guessing.
            
        OUTPUT: 
        
        A :class:`DDFunction` representing a DD-finite function or :class:`ValueError` if no relations were found.
    '''
    R = ore_ring.base()
    t = R(data)
    data = data if isinstance(data, (tuple, list)) else t.list()
    try:
        return DFinite.element(guess(t.list(), OA), init=t.list(), sequence=True)
    except ValueError:
        pass
    prec = len(data)-1
    appr_prec = max(prec//2, prec-10)
    order = 2
    T = [t]
    while order < order_bound:
        T.append(T[-1].derivative())
        M = Matrix(order, 1, T)
        B = M.minimal_approximant_basis(appr_prec)
        v = [el.list()+(appr_prec-el.degree()-1)*[0] for el in B.hermite_form()[0]]
        try:
            ops = [guess(el, ore_ring) for el in v]
        except ValueError:
            order += 1
            continue
        coeffs = [DFinite.element(ops[i],init=v[i],sequence=True) for i in range(len(ops))]
        return DDFinite.element(coeffs, init=data,sequence=True)
    raise ValueError(f"No relations found up to order {order_bound}")

#### Guessing the tangent

In [52]:
real = Tan(x)
print(f"We are guessing the equation for {repr(real)}...")
print("Time for guessing: ", end="")
%time f_tan = guess_DDFinite(real.taylor(50))
print("Time for checking: ", end="")
%time correct = (f_tan == real)
print(f"Is it correct? --> {correct}")

We are guessing the equation for tan(x)...
Time for guessing: CPU times: user 69 ms, sys: 0 ns, total: 69 ms
Wall time: 66.9 ms
Time for checking: CPU times: user 347 ms, sys: 0 ns, total: 347 ms
Wall time: 347 ms
Is it correct? --> True


#### Guessing some exponentials

In [53]:
real = Exp(Exp(x)-1)
print(f"We are guessing the equation for {repr(real)}...")
print("Time for guessing: ", end="")
%time f_exp = guess_DDFinite(real.taylor(50))
print("Time for checking: ", end="")
%time correct = (f_exp == real)
print(f"Is it correct? --> {correct}")

We are guessing the equation for exp(exp(x)-1)...
Time for guessing: CPU times: user 54.5 ms, sys: 0 ns, total: 54.5 ms
Wall time: 53 ms
Time for checking: CPU times: user 80.8 ms, sys: 0 ns, total: 80.8 ms
Wall time: 80.4 ms
Is it correct? --> True


In [54]:
real = Exp(Sin(x))
print(f"We are guessing the equation for {repr(real)}...")
print("Time for guessing: ", end="")
%time f_sin = guess_DDFinite(real.taylor(50))
print("Time for checking: ", end="")
%time correct = (f_sin == real)
print(f"Is it correct? --> {correct}")

We are guessing the equation for exp(sin(x))...
Time for guessing: 

ValueError: No relations found up to order 5

Time for checking: 

NameError: name 'f_sin' is not defined

Is it correct? --> True


In [55]:
real = Exp(Cos(x)-1)
print(f"We are guessing the equation for {repr(real)}...")
print("Time for guessing: ", end="")
%time f_cos = guess_DDFinite(real.taylor(50))
print("Time for checking: ", end="")
%time correct = (f_cos == real)
print(f"Is it correct? --> {correct}")

We are guessing the equation for exp(cos(x)-1)...
Time for guessing: 

ValueError: No relations found up to order 5

Time for checking: 

NameError: name 'f_cos' is not defined

Is it correct? --> True


What do $e^{e^x -1}$ and $\tan(x)$ have in common? If we look to the equation found for the guessing, we may obtain the answer:

In [58]:
show(f_exp)

(1:1:6)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

In [59]:
show(f_tan)

(1:1:7)DD-Function in (DD-Ring over (DD-Ring over (Univariate Polynomial Ring in x over Rational Field)))

In both cases, the equation has a $1$ in the linear term (the one corresponding to $f(x)$). This makes sense with the guessing rouine, since we are guessing using the first row of the Hermite Normal form, which will, generally, have a 1 in the first column. This is the coefficient that we will associate with the ocnstant term.

If we compute the Hermite Normal fom switching the columns, we can obtain other guesses where the corresponding coefficient for the first column will be (probably) 1.

### Current limitations

There are two main liminations to this algorithm so far:

* There is a coefficient that seems we will always get a 1. This meas we need either another approach or more information (for example on the leading coefficient of the equation).
* We do not reuse the computations of the Hermite Normal form from one order to the next. I have the impression (or feeling) that the operations should be carry over, since we get somehow a triangular matrix when computing the Hermite Normal Form. These new rows should be possible to obtain from the previous case.